In [43]:
import numpy as np
import pandas as pd

dataframe_train = pd.read_csv ("train.csv")
dataframe_train.describe()
#dataframe_train.isnull().values.any()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,target
count,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,...,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000,6963.000000
mean,-0.008188,0.046069,0.084133,-0.034631,-0.014021,1.537025,0.115259,-0.086073,0.029376,0.241310,...,-0.097963,0.025496,0.029191,-0.009160,0.003571,0.158798,0.064066,-0.205739,-0.122300,0.287233
std,3.057552,6.858400,2.695550,2.765791,7.261945,0.441206,2.762552,2.742284,2.748676,5.454805,...,2.911122,6.131952,7.884183,2.725234,2.752172,5.803769,2.779181,6.522376,6.802771,0.452503
min,-11.852801,-27.841715,-10.613443,-10.931017,-28.752811,1.000020,-10.311434,-10.075589,-10.141371,-19.277277,...,-10.638970,-28.953492,-33.201209,-11.850828,-9.478779,-20.325866,-9.926252,-28.539398,-28.483072,0.000000
25%,-2.062561,-4.317894,-1.740497,-1.896934,-4.743647,1.189318,-1.703177,-1.950722,-1.846900,-3.445588,...,-2.078737,-3.974946,-5.151041,-1.844369,-1.902487,-3.784460,-1.790703,-4.666848,-4.573261,0.000000
50%,-0.017501,-0.053759,0.084710,-0.075840,-0.091840,1.414150,0.143693,-0.063503,0.063927,0.261190,...,-0.088714,-0.008010,-0.050073,-0.007633,-0.007796,0.166533,0.027659,-0.234126,-0.093245,0.000000
75%,1.991994,4.245097,1.921483,1.777565,4.788682,1.759918,1.964735,1.786275,1.892030,3.906689,...,1.921656,4.007312,5.322672,1.840979,1.833849,4.026855,1.931042,4.296710,4.339118,1.000000
max,12.902967,34.619762,9.492417,11.881847,26.337084,2.717640,10.887146,10.071861,10.213687,21.919386,...,11.498217,26.914131,36.333892,9.047867,11.195738,22.448674,9.925644,27.139111,26.071364,1.000000


In [165]:
#Нужна кросс-валидация
x_train = dataframe_train.iloc[0:1000, :31].values.tolist() + dataframe_train.iloc[2000:3000, :31].values.tolist() + dataframe_train.iloc[4000:5000, :31].values.tolist() + dataframe_train.iloc[6000:6963, :31].values.tolist()
            
y_train = dataframe_train.iloc[0:1000, 31].values.tolist() + dataframe_train.iloc[2000:3000, 31].values.tolist() + dataframe_train.iloc[4000:5000, 31].values.tolist() + dataframe_train.iloc[6000:6963, 31].values.tolist()

np.shape(x_train), np.shape(y_train)

((3963, 31), (3963,))

In [164]:

x_test = dataframe_train.iloc[1000:2000, :31].values.tolist() + dataframe_train.iloc[3000:4000, :31].values.tolist() + dataframe_train.iloc[5000:6000, :31].values.tolist()
y_test = dataframe_train.iloc[1000:2000, 31].values.tolist() + dataframe_train.iloc[3000:4000, 31].values.tolist() + dataframe_train.iloc[5000:6000, 31].values.tolist()

np.shape(x_test), np.shape(y_test)
#/Нужна кросс-валидация

((3000, 31), (3000,))

In [129]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


inputs = keras.Input(shape=(31,))
inputs.shape

TensorShape([None, 31])

In [162]:
dense = layers.Dense(32, activation="relu")
x = dense(inputs)
outputs = layers.Dense(1, activation='tanh')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="binary_classifier_med")
model.summary()

Model: "binary_classifier_med"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 31)]              0         
                                                                 
 dense_51 (Dense)            (None, 32)                1024      
                                                                 
 dense_52 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,057
Trainable params: 1,057
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="binary_crossentropy",
),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=100)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/100
62/62 [==============================] - 1s 2ms/step - loss: 0.2951 - accuracy: 0.9553
Epoch 2/100
62/62 [==============================] - 0s 2ms/step - loss: 0.2654 - accuracy: 0.9611
Epoch 3/100
62/62 [==============================] - 0s 2ms/step - loss: 0.2196 - accuracy: 0.9659
Epoch 4/100
62/62 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.9687
Epoch 5/100
62/62 [==============================] - 0s 2ms/step - loss: 0.2159 - accuracy: 0.9727
Epoch 6/100
62/62 [==============================] - 0s 2ms/step - loss: 0.1755 - accuracy: 0.9745
Epoch 7/100
62/62 [==============================] - 0s 2ms/step - loss: 0.1721 - accuracy: 0.9765
Epoch 8/100
62/62 [==============================] - 0s 2ms/step - loss: 0.1554 - accuracy: 0.9783
Epoch 9/100
62/62 [==============================] - 0s 2ms/step - loss: 0.1334 - accuracy: 0.9811
Epoch 10/100
62/62 [==============================] - 0s 3ms/step - loss: 0.1351 - accuracy: 0.9836
Epoch 11/

62/62 [==============================] - 0s 2ms/step - loss: 0.0172 - accuracy: 0.9990
Epoch 84/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0173 - accuracy: 0.9987
Epoch 85/100
62/62 [==============================] - 0s 3ms/step - loss: 0.0171 - accuracy: 0.9990
Epoch 86/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0172 - accuracy: 0.9990
Epoch 87/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0170 - accuracy: 0.9990
Epoch 88/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0169 - accuracy: 0.9990
Epoch 89/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0170 - accuracy: 0.9990
Epoch 90/100
62/62 [==============================] - 0s 3ms/step - loss: 0.0170 - accuracy: 0.9990
Epoch 91/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0169 - accuracy: 0.9990
Epoch 92/100
62/62 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.9990
Epoch 93/100


In [160]:
dataframe_test_no_validation = pd.read_csv ("test.csv")
x_pred = dataframe_test_no_validation.values.tolist()


y_pred = model.predict(x_pred)
Y = (y_pred > .5).astype(int)
np.savetxt('pred.csv',Y, fmt='%d',header='target')
np.shape(x_pred)

123/123 [==============================] - 0s 2ms/step


(3920, 31)